# Cluster-based Vote Count Prediction (VCP) for New Images

In [1]:
# Libraries
import os
import time
import timeit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
import sklearn
from sklearn.decomposition import PCA

## Constants

In [2]:
# Paths
PROJECT_ROOT = os.path.join('..', '..')
RESULTS_FOLDER_PATH = os.path.join(PROJECT_ROOT, 'results')
SIM_MXS_FOLDER_PATH = os.path.join(RESULTS_FOLDER_PATH, 'matrices')
FEATS_FOLDER_PATH = os.path.join(RESULTS_FOLDER_PATH, 'features')
VOTES_FILE_PATH = os.path.join(RESULTS_FOLDER_PATH, 'votes_summary.csv')
IMAGES_FOLDER_PATH = os.path.join(PROJECT_ROOT, 'imgs')

In [3]:
# Execution parameters
DATA_ID = 'color_hist' # ['incv3_feats', 'incv1_feats', 'color_hist']
SIM_METRIC_ID = 'euclid' # ['euclid', 'cosine']
FULL_ID = DATA_ID + '_' + SIM_METRIC_ID
VOTE_PRT_TYPE = 'avg-votes' # ['avg-votes']
FEAT_PRT_TYPE = 'avg-feats' # ['avgf-feats', 'pca']
VOTE_PRED_TYPE = 'prt-based' # ['prt-based', 'prtless', 'pca-proj']
TIMER_ID = 'timeit' # ['timeit', 'time']
K_RANGE = [1, 3, 5, 7, 10] # list-like
PERF_METRIC_ID = 'rmse' # ['rmse', 'manhattan']
OVERWRITE_CL_PARAMS = True
SEED = 42 # Highly recommended to leave it at 42

In [4]:
# Additional configuration
tictoc = timeit.default_timer if TIMER_ID == 'timeit' else time.time

## Loading Data

In [5]:
def gen_file_paths(feat_type, sim_type, feats_folder_path, sim_mx_folder_path):
    # Gen features file path
    feats_file_name = f'{feat_type}.csv'
    feats_file_path = os.path.join(feats_folder_path, feats_file_name)
    # Gen. sim. matrix. file path
    sim_mx_file_name = f'{feat_type}_{sim_type}_sim_matrix.csv'
    sim_mx_file_path = os.path.join(sim_mx_folder_path, sim_mx_file_name)
    return feats_file_path, sim_mx_file_path

In [6]:
# Features and similarity matrix files
FEATS_FILE_PATH, SIM_MX_FILE_PATH = gen_file_paths(DATA_ID, SIM_METRIC_ID, FEATS_FOLDER_PATH, SIM_MXS_FOLDER_PATH)

#### Features

In [7]:
feats_df = pd.read_csv(FEATS_FILE_PATH, index_col=0)
feats_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
1222__pool_table__0.9999995.jpg,178,51,43,49,37,40,54,57,57,54,...,8,5,12,9,13,14,12,12,7,51
1328__coil__0.99999607.jpg,47,39,66,118,112,134,143,164,194,230,...,97,114,127,188,211,172,121,90,61,186
134__zebra__0.9999949.jpg,0,0,1,1,4,4,7,5,12,8,...,34,17,40,14,25,12,2,4,2,13


#### Data (Sim. Matrix between images)

In [8]:
sim_mx_df = pd.read_csv(SIM_MX_FILE_PATH, index_col=0)
sim_mx_df.head(3)

,1222__pool_table__0.9999995.jpg,1328__coil__0.99999607.jpg,134__zebra__0.9999949.jpg,2377471__pizza__0.9999988.jpg,2377620__zebra__0.9999882.jpg,2377698__zebra__0.9999999.jpg,2378170__zebra__0.9999902.jpg,2378358__park_bench__0.99999833.jpg,2378523__banana__0.99999785.jpg,2379086__zebra__0.9999975.jpg,...,2417881__zebra__0.9999945.jpg,2417938__banana__0.9999944.jpg,4099__pool_table__0.9999945.jpg,4339__manhole_cover__0.99999416.jpg,4534__viaduct__0.9999877.jpg,4573__barrel__0.9999974.jpg,4673__triumphal_arch__0.9999893.jpg,576__gondola__0.9999993.jpg,577__gondola__0.9999962.jpg,691__cheetah__0.99999213.jpg
1222__pool_table__0.9999995.jpg,0.000000,6285.621051,14593.537679,5772.380618,11441.225721,11880.166834,9123.238350,5983.605101,7356.368941,8848.581129,...,10007.829435,8019.789274,10733.466448,9367.552722,8243.123316,7766.734835,7264.524073,9446.944797,7846.279883,8571.525069
1328__coil__0.99999607.jpg,6285.621051,0.000000,12651.202235,4392.810718,9290.190310,10232.351538,6772.767824,4410.541237,7820.760705,6133.401177,...,7638.769273,5194.897304,8131.912321,6862.227918,6088.735008,7629.691999,3859.095749,7329.847611,5987.893787,6005.235050
134__zebra__0.9999949.jpg,14593.537679,12651.202235,0.000000,13183.485427,15001.600315,14800.066284,12944.626144,14127.997947,13790.634141,13337.084014,...,13134.598129,13174.048960,11249.446564,13218.859331,11597.730726,14856.417065,11823.161168,13457.201195,13181.741160,12538.407634


#### Votes

In [9]:
votes_df = pd.read_csv(VOTES_FILE_PATH, index_col=0)
votes_df.head(3)

,ig,lime,xrai,anchor,best
1222__pool_table__0.9999995.jpg,12,13,3,1,lime
1328__coil__0.99999607.jpg,17,4,3,2,ig
134__zebra__0.9999949.jpg,14,1,8,2,ig


Here's a sanity check for vote proportion in our the dataset. In the original XAI-CBR paper, vote proportion was like this:
- IG: 45%
- XRAI: 30%
- LIME: 18%
- ANCHOR: 7%

Also, IG was the most voted technique, at least by hard voting aggregation, with a majority of 62% images.


In [10]:
votes_per_technique = votes_df[['ig', 'xrai', 'lime', 'anchor']].sum()
total_votes = votes_per_technique.sum()
votes_per_technique / total_votes

ig        0.488315
xrai      0.271713
lime      0.183467
anchor    0.056505
dtype: float64

There's a slight variation of these proportions with respect to ones presented in the paper. It seems like some votes from XRAI and ANCHOR techniques drifted out to the IG technique. We'll check this out later, this should not be of great importance in the experiments of this notebook.

## Data Preprocessing

In [11]:
X = sim_mx_df.values # Values from sim. matrix
X_names = sim_mx_df.index.values # Names of every image
y = votes_df.values[:, :4] # Vote count for each imae
best = votes_df.values[:, -1] # Most voted technique for each image

In [12]:
print(X.shape, X_names.shape, y.shape, best.shape)

(198, 198) (198,) (198, 4) (198,)


#### Instance deletion
Stratified Subsampling cannot be performed onto the dataset because only one instance is best explained with ANCHOR. Due to the very small importance of that instance in the dataset, we will continue without that instance (i.e. we will find that instance and remove it from the dataset).

In [13]:
# At what index is the anchor instance located?
anchor_idxs = np.argwhere(best == 'anchor')[0]
anchor_idxs

array([155], dtype=int64)

In [14]:
# What's the name of that image and its associated technique?
X_names[anchor_idxs], best[anchor_idxs]

(array(['2411942__zebra__0.99999654.jpg'], dtype=object),
 array(['anchor'], dtype=object))

In [15]:
# Delete that instance from all data partitions (X, y, etc.)
X = np.delete(X, anchor_idxs, axis=0)
X = np.delete(X, anchor_idxs, axis=1) # Twice in sim. matrix (both rows and columns)
X_names = np.delete(X_names, anchor_idxs, axis=0)
y = np.delete(y, anchor_idxs, axis=0)
best = np.delete(best, anchor_idxs, axis=0)

In [16]:
print(X.shape, X_names.shape, y.shape, best.shape)

(197, 197) (197,) (197, 4) (197,)


## Splitting and Fold Creation

In [17]:
from sklearn.model_selection import StratifiedShuffleSplit as SSS
from sklearn.model_selection import ShuffleSplit as SS

In [18]:
# Change this constant to toogle stratified sampling on/off
STRATIFIED = True

In [19]:
# Perform split
splitter = None
if STRATIFIED: splitter = SSS(n_splits=5, test_size=0.2, random_state=42)
else: splitter = SS(n_splits=5, test_size=0.2, random_state=42)
splits = splitter.split(X, best)
splits = list(splits)

In [20]:
splits[0]

(array([192, 147, 177,  11, 140,  51, 127, 118, 172, 191,  62, 124, 115,
         80, 190, 142,  92,  69,  25,  14,  42,   3, 185,  90,  10,  76,
        176, 114,  44,  98, 166, 121,  79, 170,   1, 183,  28,  31, 155,
         75, 156, 101, 171,  13, 110, 122,  38,  27, 136,  20,   6,  56,
         35,  59, 139,  33,  78,  82,  21, 167, 117,  12,  49,  15,   5,
        152, 132,  81,  61, 163, 175,  91,   7, 174, 135,  74, 193, 129,
         60,  96,  50, 161, 159, 145, 126,  19,  65, 188,  73,  89, 133,
        179,  40,  86, 112,  26, 168, 189, 149,  94, 194,  18, 138, 169,
        102,  97,  71, 130,  53,  99, 148, 154,   8,  34, 182, 105,  55,
         95, 153,  72, 144,  77,  52,  30,   9,  37,   4,  93, 128, 137,
        195, 160, 111,  45, 164, 151,  29,  48,  70,  43,  57, 157,  39,
        141,  85, 150,  67,   0,  47, 113,  32,  17, 131, 180,  66, 100,
        186], dtype=int64),
 array([ 54, 187, 103,  23, 104, 108, 181,  64, 109, 134,  16, 146,   2,
        116, 106, 119, 

## Clustering (using DBSCAN)

In [21]:
clusterable_params = []

In [22]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

In [23]:
def get_sim_mx_subset(sim_mx_values, filter_idxs):
    return sim_mx_values.take(filter_idxs, axis=0).take(filter_idxs, axis=1)

In [24]:
def fit_dbscan_sim_mx(data, min_samples, eps_values, 
               min_no_clusters=5, max_no_clusters=np.inf,
               min_clust_instances=None, min_clust_instances_pct=0.85,
               max_clust_instances=np.inf):
    '''Performs several DBSCAN clustering runs according to a parameter search grid (using m and epsilon).
    Also, verifies different clustering conditions and calculates sil. score of all valid clustering runs.'''
    # Condition precalculation
    if min_clust_instances_pct: # If % was defined
        min_clust_instances = round(data.shape[0] * min_clust_instances_pct)
    elif not min_clust_instances: # Else, if nominal amount was not specified
        min_clust_instances = 100
    # Code
    scores, clusters, instances = [], [], []
    for m in min_samples:
        row_scores, row_clusters, row_instances = [], [], []
        for e in eps_values:
            db = DBSCAN(min_samples=m, eps=e, metric='precomputed').fit(data)
            # Get only non anomalous instances and indices
            non_a = db.labels_ != -1 # [False, ..., False] if all are outliers
            non_a_idxs = np.argwhere(non_a==True)
            non_a_idxs = non_a_idxs.reshape(non_a_idxs.shape[0]) # flatten matrix to 1D
            # Calculate conditions
            n_clusters = len(np.unique(db.labels_[non_a])) # 0 if all are outliers
            n_instances = len(db.labels_[non_a]) # 0 if all are outliers
            # Apply conditions
            valid_n_clusters = n_clusters >= min_no_clusters and n_clusters <= max_no_clusters
            valid_n_cl_instances = n_instances >= min_clust_instances and n_instances <= max_clust_instances
            if (valid_n_clusters and valid_n_cl_instances):
                # Calculate silhouette score without noise points (i.e. anomalous instances)
                non_a_sim_mx = get_sim_mx_subset(data, non_a_idxs)
                score = silhouette_score(non_a_sim_mx, db.labels_[non_a], metric='precomputed')
            else:
                score = None
            # Store results
            row_scores.append(score)
            row_clusters.append(n_clusters)
            row_instances.append(n_instances)
        # Store row results
        scores.append(row_scores)
        clusters.append(row_clusters)
        instances.append(row_instances)
    # Prepare and return values
    ms_axis = pd.Index(min_samples, name='Min_samples')
    eps_axis = pd.Index(eps_values, name='Epsilon')
    df_scores = pd.DataFrame(scores, index=ms_axis, columns=eps_axis)
    df_clusters = pd.DataFrame(clusters, index=ms_axis, columns=eps_axis)
    df_instances = pd.DataFrame(instances, index=ms_axis, columns=eps_axis)
    return df_scores, df_clusters, df_instances

In [25]:
def print_results(m, eps, scores_df, instances_df, clusters_df):
    '''Given clustering results and specific values for m and epsilon parameters,
    displays further info. about the clustering obtained using those parameters.
    For interactive and visual use only'''
    score = round(scores_df.loc[m][eps], 4)
    instances = instances_df.loc[m][eps]
    clusters = clusters_df.loc[m][eps]
    print(f'DBSCAN using parameters m={m} and eps={eps} yields the next clustering results:')
    print()
    print(f'- Sil. score: {score}')
    print(f'- {instances} clustered instances into {clusters} clusters')
    print(f'- Avg. of {round(instances/clusters, 2)} instances per cluster')

In [26]:
# What's the equivalence in instances of different clustered proportions of the dataset?
for i in np.arange(0.7, 0.95, 0.05):
    print(f'{round(i, 2)}% : about {round(X[splits[0][0]].shape[0] * i, 2)} instances')

0.7% : about 109.9 instances
0.75% : about 117.75 instances
0.8% : about 125.6 instances
0.85% : about 133.45 instances
0.9% : about 141.3 instances


#### Split #0

In [27]:
X_split_0 = get_sim_mx_subset(X, splits[0][0])
X_split_0.shape

(157, 157)

In [28]:
# dfs, dfc, dfi = fit_dbscan_sim_mx(X_split_0, range(2, 8), range(10, 18))
# dfs

In [29]:
# print_results(2, 11, dfs, dfi, dfc)

In [30]:
# clusterable_params.append([2, 11, 0])

#### Split #1

In [31]:
X_split_1 = get_sim_mx_subset(X, splits[1][0])
X_split_1.shape

(157, 157)

In [32]:
# dfs, dfc, dfi = fit_dbscan_sim_mx(X_split_1, range(2, 8), range(10, 18))
# dfs

In [33]:
# print_results(2, 11, dfs, dfi, dfc)

In [34]:
# clusterable_params.append([2, 11, 1])

#### Split #2

In [35]:
X_split_2 = get_sim_mx_subset(X, splits[2][0])
X_split_2.shape

(157, 157)

In [36]:
# dfs, dfc, dfi = fit_dbscan_sim_mx(X_split_2, range(2, 8), range(10, 18))
# dfs

In [37]:
# print_results(2, 11, dfs, dfi, dfc)

In [38]:
# clusterable_params.append([2, 11, 2])

#### Split #3

In [39]:
X_split_3 = get_sim_mx_subset(X, splits[3][0])
X_split_3.shape

(157, 157)

In [40]:
# dfs, dfc, dfi = fit_dbscan_sim_mx(X_split_3, range(2, 8), range(10, 18))
# dfs

In [41]:
# print_results(2, 11, dfs, dfi, dfc)

In [42]:
# clusterable_params.append([2, 11, 3])

#### Split #4

In [43]:
X_split_4 = get_sim_mx_subset(X, splits[4][0])
X_split_4.shape

(157, 157)

In [44]:
# dfs, dfc, dfi = fit_dbscan_sim_mx(X_split_4, range(2, 8), range(10, 18))
# dfs

In [45]:
# print_results(2, 11, dfs, dfi, dfc)

In [46]:
# clusterable_params.append([2, 11, 4])

#### Clusterable parameters for each split
They have been stablished before the notebook ran. Clustering section was not removed due to compatibility issues, but clustering parameters are already defined , no matter what the clustering results are.

In [47]:
def get_clusterable_params(feat_id, sim_metric_id):
    cl_params = {
        'incv3_feats_euclid': [[2, 11, 0], [2, 11, 1], [2, 11, 2], [2, 11, 3], [2, 11, 4]],
        'incv3_feats_cosine': [[2, 0.2, 0], [2, 0.2, 1], [3, 0.25, 2], [2, 0.2, 3], [2, 0.2, 4]],
        'color_hist_euclid': [[2, 4800, 0], [2, 4900, 2], [2, 7600, 3], [2, 6400, 4]],
        'color_hist_cosine': [[3, 0.16, 0], [2, 0.2, 1], [4, 0.16, 2], [3, 0.2, 3], [2, 0.16, 4]]
    }
    return cl_params[f'{feat_id}_{sim_metric_id}']

In [48]:
if OVERWRITE_CL_PARAMS:
    clusterable_params = get_clusterable_params(DATA_ID, SIM_METRIC_ID)

In [49]:
# [m, epsilon, split_idx]
clusterable_params

[[2, 4800, 0], [2, 4900, 2], [2, 7600, 3], [2, 6400, 4]]

## Clustering Results

In [50]:
def get_indiv_clustering_results(params, split, image_names):
    '''Returns a dictionary mapping the name of an image with an integer
    representing the cluster it belongs to'''
    # Fetch data
    train_idxs = split[0]
    sim_mx_subset = get_sim_mx_subset(X, train_idxs)
    img_names = image_names[train_idxs]
    # Perform clustering
    dbscan = DBSCAN(min_samples=params[0], eps=params[1], metric='precomputed')
    dbscan = dbscan.fit(sim_mx_subset)
    # Generate {img_name : label} mapping
    name_label_map = {name: label for name, label in zip(img_names, dbscan.labels_)}
    return name_label_map

def get_global_clustering_results(params_set, splits, image_names):
    '''Returns a dictionary mapping the split index of every parameter set
    in the 'params_set' arg. with the clustering results generated with that parameter set'''
    results = {}
    for params in params_set:
        split_idx = params[2]
        # Create { split_idx: cluster_labels} pair
        results[split_idx] = get_indiv_clustering_results(params, splits[split_idx], image_names)
    return results

In [51]:
global_cl_results = get_global_clustering_results(clusterable_params, splits, X_names)

In [52]:
global_cl_results

{0: {'4573__barrel__0.9999974.jpg': -1,
  '2411372__parking_meter__0.999995.jpg': -1,
  '2415910__zebra__0.9999962.jpg': 0,
  '2380017__zebra__0.9999995.jpg': 0,
  '2410410__ski__0.99999356.jpg': -1,
  '2387305__traffic_light__1.0.jpg': -1,
  '2408884__zebra__0.9999913.jpg': -1,
  '2406581__zebra__0.9999939.jpg': 0,
  '2415102__zebra__0.9999876.jpg': -1,
  '4534__viaduct__0.9999877.jpg': 0,
  '2391862__broccoli__0.99999714.jpg': 0,
  '2408592__goose__0.999998.jpg': 0,
  '2405479__traffic_light__0.9999939.jpg': -1,
  '2396034__remote_control__0.9999856.jpg': 0,
  '4339__manhole_cover__0.99999416.jpg': 0,
  '2410779__parking_meter__0.99999917.jpg': 0,
  '2401383__slug__0.9999933.jpg': 0,
  '2392579__zebra__0.9999969.jpg': 1,
  '2382183__pizza__0.99998593.jpg': 0,
  '2380319__broccoli__0.9999957.jpg': 0,
  '2385461__zebra__0.99998415.jpg': 0,
  '2377471__pizza__0.9999988.jpg': 0,
  '2417421__parking_meter__0.9999999.jpg': 0,
  '2401217__traffic_light__0.9999895.jpg': -1,
  '2379489__parki

In [53]:
# Sanity check: Number of elements should be the same as clusters detected in clustering phase
for split_idx in global_cl_results.keys(): print(len(np.unique(list(global_cl_results[split_idx].values())))-1)

2
2
2
2


It was 15, 16, 15, 15, 15. Clustering results is fine.

## Clustering Prototypes

In our experiment, we want to predict the vote count for a new image, based on the proximity it has to the avaliable clusters. These clusters are composed of many data points, so the proximity of a new data point to a cluster can be measured in different ways, like taking the distance between the new point and the nearest clustered point in the dataset.   
However, this approach can be biased when new poins get associated to the cluster taking in account the nearest point of a cluster instead of the overall position of a cluster. To avoid this, for each cluster we calculate a "prototype", a data point which is the centroid of all the data points in a cluster. This way, we can measure the distance to the general position of a cluster in a more confident way.

### Vote prototypes (Solution of cases)

In [54]:
# TODO: SI there another to calculate vote prototypes than to just average the votes
# casted for every image in a specific cluster? 

def gen_indiv_vote_prototypes(cl_result, votes_df, vote_prt_type, ignore_noise=True):
    '''Maps the integer labels of every cluster with its vote prototype'''
    # Separate image votes according to the clusters they belong to
    votes_by_cluster = {}
    for img_name, cl_idx in cl_result.items():
        if ignore_noise and cl_idx == -1: continue # ignore noise cluster
        img_votes = votes_df.loc[img_name].values[:-1] # fetch vote columns (but not "best technique" column)
        if cl_idx not in votes_by_cluster.keys(): votes_by_cluster[cl_idx] = [img_votes]
        else: votes_by_cluster[cl_idx].append(img_votes)
    # For each cluster, calculate their vote prototype
    vote_prts_by_cluster = {}
    for cl_idx, cl_votes in votes_by_cluster.items():
        unrounded_prt = np.average(np.array(cl_votes, 'uint8'), axis=0) # load with uint8 to prevent unwanted floating points
        # Create { cluster_idx : vote_prototype } pairs
        vote_prts_by_cluster[cl_idx] = np.array(np.round(unrounded_prt), 'int')
    return vote_prts_by_cluster
    
def get_global_vote_prototypes(cl_results, votes_df, vote_prt_type, ignore_noise=True):
    '''Maps indices of folds/splits used to generate clusterings with the vote prototypes
    of the different clusters generated'''
    global_vote_prototypes = {}
    for i, cl_result in cl_results.items():
        # Create { split_idx : vote_prototypes_per_cluster } pairs
        global_vote_prototypes[i] = gen_indiv_vote_prototypes(cl_result, votes_df, vote_prt_type, ignore_noise)
    return global_vote_prototypes

### Feature Prototypes (Description of cases)

#### Default Feature Prototyping (Average of features)

In [55]:
def gen_averaged_feat_prototypes(cl_result, feats_df, ignore_noise=True):
    # Separate image votes according to the clusters they belong to
    feats_by_cluster = {}
    for img_name, cl_idx in cl_result.items():
        if ignore_noise and cl_idx == -1: continue # ignore noise cluster
        img_feats = feats_df.loc[img_name].values # fetch features of image
        if cl_idx not in feats_by_cluster.keys(): feats_by_cluster[cl_idx] = [img_feats]
        else: feats_by_cluster[cl_idx].append(img_feats)
    # For each cluster, calculate their feature prototype
    feat_prts_by_cluster = {}
    for cl_idx, cl_feats in feats_by_cluster.items():
        unrounded_prt = np.average(np.array(cl_feats), axis=0)
        feat_prts_by_cluster[cl_idx] = unrounded_prt
        # print(feat_prts_by_cluster[cl_idx])
    return feat_prts_by_cluster

#### PCA Feature Prototyping

In [56]:
def gen_pca_feat_prototypes(cl_result, images_folder_path, ignore_noise=True):
    # For each cluster:
        # Load images (using img_name in cl_result and images folder path)
        # Flatten images and condense them into a matrix
        # Apply PCA over the matrix
        # Store the trained PCA object as a feature prototype with a "cluster_idx" key
    # return pca_objs_per_cluster
    pass

#### General code

In [57]:
def gen_indiv_feat_prototypes(cl_result, feat_prt_type, ignore_noise=True, **kwargs):
    '''Maps the integer labels of every cluster with its feature prototype'''
    # Execute feature prototyping code according to selected prototyping type
    if feat_prt_type == 'avg-feats':
        feat_prts_per_cluster = gen_averaged_feat_prototypes(cl_result,
                                                             feats_df = kwargs['feats_df'],
                                                             ignore_noise=ignore_noise)
    elif feat_prt_type == 'pca':
        feat_prts_per_cluster = gen_pca_feat_prototypes(cl_result,
                                                        images_folder_path = kwargs['images_folder_path'],
                                                        ignore_noise=ignore_noise)
    else:
        raise Exception(f'Unknown feature prot. type : {feat_prt_type}')
    return feat_prts_per_cluster
    
def get_global_feat_prototypes(cl_results, feat_prt_type, ignore_noise=True, **kwargs):
    '''Maps indices of folds/splits used to generate clusterings with the feat. prototypes
    of the different clusters generated'''
    global_feat_prototypes = {}
    for split_idx, cl_result in cl_results.items():
        # Create { split_idx : feat_prototypes_per_cluster } pairs
        global_feat_prototypes[split_idx] = gen_indiv_feat_prototypes(cl_result, feat_prt_type, ignore_noise, **kwargs)
    return global_feat_prototypes

In [58]:
# Warning: This code access variables defined previously without referencing them in its arguments.
def get_prototypes(vote_prt_type, feat_prt_type):
    # Calculate vote prototypes
    vote_prts = None
    if vote_prt_type=='avg-votes':
        vote_prts = get_global_vote_prototypes(global_cl_results, votes_df, vote_prt_type)
    else: raise Exception(f'Unknown vote prot. type : {vote_prt_type}')
    # Calculate feature prototypes
    feat_prts = None
    if feat_prt_type=='avg-feats':
        feat_prts = get_global_feat_prototypes(global_cl_results, FEAT_PRT_TYPE, feats_df=feats_df)
    elif feat_prt_type=='pca':
        feat_prts = get_global_feat_prototypes(global_cl_results, FEAT_PRT_TYPE, images_folder_path=IMAGES_FOLDER_PATH)
    else: raise Exception(f'Unknown feature prot. type : {feat_prt_type}')
    # Return prototypes
    return vote_prts, feat_prts

### Calculate both prototypes

In [59]:
global_vote_prototypes, global_feat_prototypes = get_prototypes(VOTE_PRT_TYPE, FEAT_PRT_TYPE)

In [60]:
# Sanity check: No. of elements should be the same as no. of clusters detected in clustering phase
global_vote_prototypes[3]

{0: array([7, 3, 4, 1]), 1: array([6, 2, 4, 1])}

In [61]:
# Sanity check: No. of elements should be the same as no. of clusters detected in clustering phase
if global_feat_prototypes is not None:
    print(global_feat_prototypes[3]) # float64

{0: array([362.13286713, 182.43356643, 149.57342657, 155.60839161,
       143.83916084, 131.06993007, 129.04195804, 129.1958042 ,
       132.76223776, 133.14685315, 143.44055944, 152.23776224,
       158.85314685, 168.3986014 , 172.84615385, 173.47552448,
       175.74125874, 180.62937063, 185.52447552, 186.6013986 ,
       188.95104895, 190.02097902, 187.04895105, 182.55944056,
       184.99300699, 185.75524476, 183.96503497, 186.32867133,
       188.3986014 , 188.98601399, 188.46153846, 183.71328671,
       181.1048951 , 180.53846154, 180.55244755, 178.35664336,
       177.77622378, 178.91608392, 181.07692308, 181.35664336,
       182.06993007, 178.7972028 , 178.61538462, 178.65034965,
       178.03496503, 175.36363636, 174.79020979, 177.05594406,
       173.52447552, 172.39160839, 169.38461538, 170.06993007,
       169.88811189, 169.54545455, 169.04195804, 166.37762238,
       169.85314685, 168.76923077, 170.05594406, 170.6993007 ,
       171.32867133, 172.67832168, 170.21678322, 17

## Vote Count Prediction (using feat prototypes and vote prototypes)

#### Helper functions

In [62]:
def dist_to_prt(p1, p2, metric):
    if metric=='euclid': return np.linalg.norm(p1-p2)
    if metric=='cosine': return 1 - (np.dot(p1, p2)/(np.linalg.norm(p1) * np.linalg.norm(p2)))
    else: raise Exception(f'Unknown metric type : {metric}')

### Prototype-based VCP

In [63]:
def get_distances_to_feat_prts(test_img_name, feats_df, gbl_feat_prts, split_idx, metric, **kwargs):
    # Prepare data
    test_feats = feats_df.loc[test_img_name].values
    feat_prts = gbl_feat_prts[split_idx]
    # Calculate distances
    distances_per_feat_prt = {}
    for prt_idx, prt_feats in feat_prts.items():
        distances_per_feat_prt[prt_idx] = dist_to_prt(test_feats, prt_feats, metric)
    return distances_per_feat_prt

### PCA-based VCP

In [64]:
def get_pca_proj_distances(test_img_name, images_folder_path, gbl_feat_prts, split_idx, **kwargs):
    pass

### Prototypeless VCP

In [65]:
def get_img_idxs_per_cluster(cl_results, ignore_noise=True):
    img_idxs_per_cluster = {}
    for img_name, cl_idx in cl_results.items():
        if cl_idx==-1 and ignore_noise: continue # ignore noise cluster
        img_idx = np.argwhere(X_names == img_name)[0][0]
        if cl_idx not in img_idxs_per_cluster.keys():
            img_idxs_per_cluster[cl_idx] = [img_idx]
        else:
            img_idxs_per_cluster[cl_idx].append(img_idx)
    return img_idxs_per_cluster

def get_intercluster_distances(test_img_name, feats_df, cl_results, split_idx, metric, **kwargs):
    pass

### Main code

In [66]:
def get_nearest_prototypes_indices(dist_to_prototypes, k):
    '''Given a dictionary between feature prototypes indices and the distance to a point,
    return indices of nearest prototypes (i.e. those with the lowest distances).
    Indices of prototypes are NOT guaranteed to be ordered in terms of closeness'''
    # Return existing prototypes if k is higher than no. of prototypes
    if k >= len(dist_to_prototypes): return list(dist_to_prototypes.keys())
    # Else, find nearest prototypes
    nearest_prts_idxs = []
    # Iterate k times...
    for i in range(k):
        nearest_prt_idx, min_dist = None, np.inf
        # ...searching the next nearest prototype
        for prt_idx, dist in dist_to_prototypes.items():
            if prt_idx in nearest_prts_idxs: continue # ignore prev. found nearest prototypes
            if dist < min_dist: nearest_prt_idx, min_dist = prt_idx, dist
        nearest_prts_idxs.append(nearest_prt_idx)
    return nearest_prts_idxs

def gen_prediction(test_img_name, vote_prts, k, vote_pred_type, **kwargs):
    # Calculate distances to each cluster/prototype depending on vote_pred_type
    if vote_pred_type == 'prt-based':
        dist_to_prototypes = get_distances_to_feat_prts(test_img_name, **kwargs)
    elif vote_pred_type == 'pca-based':
        dist_to_prototypes = get_pca_proj_distances(test_img_name, **kwargs)
    elif vote_pred_type == 'prtless':
        dist_to_prototypes = get_intercluster_distances(test_img_name, **kwargs)
    else:
        raise Exception(f'Unknown vote prediction type : {vote_pred_type}')
    # Find k nearest prototypes
    kn_prototypes_idxs =  get_nearest_prototypes_indices(dist_to_prototypes, k)
    # Aggregate the vote prototypes of the clusters associated with those distances
    nearest_vote_prototypes = [vote_prts[kn_prt_idx] for kn_prt_idx in kn_prototypes_idxs]
    unrounded_vcp = np.average(np.array(nearest_vote_prototypes), axis=0)
    rounded_vcp = np.round(unrounded_vcp, 0).astype(np.int8)
    vote_count_prediction = rounded_vcp
    return vote_count_prediction

In [67]:
def get_indiv_vote_predictions(split, vote_prts, image_names, k, vote_pred_type, **kwargs):
    '''Maps the name of every test image with its generated vote count prediction'''
    vote_predictions = {}
    query_times = {}
    # Prepare data
    test_idxs = split[1]
    # For each test image...
    for test_img_idx in test_idxs:
        # Register query computation time
        start = tictoc()
        # Generate and store prediction (along with the image's name)
        kwargs['image_names'] = image_names
        test_img_name = image_names[test_img_idx]
        vote_count_prediction = gen_prediction(test_img_name, vote_prts, k, vote_pred_type, **kwargs)
        vote_predictions[test_img_name] = vote_count_prediction
        # Calculate and store query computation time
        end = tictoc()
        query_time = end - start
        query_times[test_img_name] = query_time
    return vote_predictions, query_times

def get_global_vote_predictions(splits, gbl_vote_prts, image_names, k, vote_pred_type, **kwargs):
    '''Maps indices of test folds/splits with the vote count predictions generated'''
    global_vote_predictions = {}
    global_query_times = {}
    for split_idx in gbl_vote_prts.keys():
        # Generate predictions and query times.
        kwargs['split_idx'] = split_idx # Some functions use this parameter
        vote_preds, query_times = get_indiv_vote_predictions(splits[split_idx], gbl_vote_prts[split_idx], image_names, k, vote_pred_type, **kwargs)
        # Create { split_idx: vote_predictions } and { split_idx : query_times } pairs
        global_vote_predictions[split_idx] = vote_preds
        global_query_times[split_idx] = query_times
    return global_vote_predictions, global_query_times

In [68]:
# Warning: This code access variables defined previously without referencing them in its arguments.
def get_predictions(k, vote_pred_type, sim_metric_id, **kwargs):
    predictions = None
    if vote_pred_type == 'prt-based':
        predictions = get_global_vote_predictions(splits, global_vote_prototypes, X_names, k, vote_pred_type,
                                                  feats_df=feats_df, gbl_feat_prts=global_feat_prototypes, metric=sim_metric_id)
    elif vote_pred_type == 'pca-based':
        predictions = get_global_vote_predictions(splits, global_vote_prototypes, X_names, k, vote_pred_type,
                                                 images_folder_path=IMAGES_FOLDER_PATH, gbl_pca_objs=global_feat_prototypes)
    elif vote_pred_type == 'prtless':
        predictions = get_global_vote_predictions(splits, global_vote_prototypes, X_names, k, vote_pred_type,
                                                  feats_df=feats_df, gbl_cl_results=global_cl_results, metric=sim_metric_id)
    else:
        raise Exception(f'Unknown vote prediction type : {vote_pred_type}')
    return predictions

##### A little function to evaluate different values for k

In [69]:
def batch_get_predictions(k_range, **kwargs):
    batch_predictions, batch_query_times = {}, {}
    for k in k_range:
        preds, query_times = get_predictions(k, **kwargs)
        batch_predictions[k] = preds
        batch_query_times[k] = query_times
    return batch_predictions, batch_query_times

### Generating predictions

In [70]:
# Sanity check / Remainder of constants
print(VOTE_PRED_TYPE, SIM_METRIC_ID)

prt-based euclid


In [71]:
k_range = K_RANGE # Substitute if neccesary
batch_results = batch_get_predictions(k_range, vote_pred_type=VOTE_PRED_TYPE, sim_metric_id=SIM_METRIC_ID)
batch_global_predictions, batch_global_query_times = batch_results

In [72]:
# For k=5, in the test split #0, show generated predictions
batch_global_predictions[5][0]

{'2388889__hotdog__0.99999714.jpg': array([6, 2, 5, 1], dtype=int8),
 '2417881__zebra__0.9999945.jpg': array([6, 2, 5, 1], dtype=int8),
 '2403403__banana__0.9999926.jpg': array([6, 2, 5, 1], dtype=int8),
 '2381941__zebra__0.9999914.jpg': array([6, 2, 5, 1], dtype=int8),
 '2403741__zebra__0.99999523.jpg': array([6, 2, 5, 1], dtype=int8),
 '2404281__zebra__0.999998.jpg': array([6, 2, 5, 1], dtype=int8),
 '2416627__zebra__0.9999987.jpg': array([6, 2, 5, 1], dtype=int8),
 '2391964__flamingo__1.0.jpg': array([6, 2, 5, 1], dtype=int8),
 '2404583__umbrella__0.99999297.jpg': array([6, 2, 5, 1], dtype=int8),
 '2409637__four-poster__0.99999464.jpg': array([6, 2, 5, 1], dtype=int8),
 '2380669__parking_meter__0.9999993.jpg': array([6, 2, 5, 1], dtype=int8),
 '2411196__crane__0.9999995.jpg': array([6, 2, 5, 1], dtype=int8),
 '134__zebra__0.9999949.jpg': array([6, 2, 5, 1], dtype=int8),
 '2405905__traffic_light__0.99999535.jpg': array([6, 2, 5, 1], dtype=int8),
 '2404127__zebra__0.9999933.jpg': arra

In [73]:
# For k=5, in the test split #0, show query times
batch_global_query_times[5][0]

{'2388889__hotdog__0.99999714.jpg': 0.00036869999999922243,
 '2417881__zebra__0.9999945.jpg': 0.00032930000000064297,
 '2403403__banana__0.9999926.jpg': 0.00032939999999825886,
 '2381941__zebra__0.9999914.jpg': 0.0006221999999986849,
 '2403741__zebra__0.99999523.jpg': 0.0004629999999998802,
 '2404281__zebra__0.999998.jpg': 0.0003409000000011986,
 '2416627__zebra__0.9999987.jpg': 0.0003244000000002245,
 '2391964__flamingo__1.0.jpg': 0.00038780000000215864,
 '2404583__umbrella__0.99999297.jpg': 0.00046809999999908314,
 '2409637__four-poster__0.99999464.jpg': 0.00033120000000153027,
 '2380669__parking_meter__0.9999993.jpg': 0.0003911000000016429,
 '2411196__crane__0.9999995.jpg': 0.00032919999999947436,
 '134__zebra__0.9999949.jpg': 0.0003274000000033084,
 '2405905__traffic_light__0.99999535.jpg': 0.000335700000000827,
 '2404127__zebra__0.9999933.jpg': 0.00031809999999765637,
 '2406857__zebra__0.9999894.jpg': 0.0003223000000005527,
 '2414277__zebra__0.9999908.jpg': 0.0003378000000004988,


## Metric Evaluation

### Vote Accuracy (RMSE, Manhattan, etc...)

In [74]:
def calc_vote_distance(p1, p2, metric):
    if metric == 'rmse': return np.sum(np.square(p1 - p2))
    elif metric == 'manhattan': return np.sum(np.abs(p1 - p2))
    else: raise Exception(f'Unknown performance metric : {metric}')
        
def eval_indiv_vote_predictions(vote_predictions, perf_metric):
    vote_distances = []
    for img_name, vote_pred in vote_predictions.items():
        # Fetch real votes and compare with vote predictions
        real_votes = votes_df.loc[img_name].values[:4]
        distance = calc_vote_distance(real_votes, vote_pred, perf_metric)
        vote_distances.append(distance)
    vote_distances = np.array(vote_distances)
    if perf_metric=='rmse': metrics = {'rmse': round(np.sqrt(np.average(vote_distances)), 2)}
    elif perf_metric=='manhattan': metrics = {'manhattan': (np.average(vote_distances), 2)}
    else: raise Exception(f'Unknown performance metric : {perf_metric}')
    return metrics

def eval_global_vote_predictions(global_vote_predictions, perf_metric):
    global_metrics = {}
    # Calculate metrics for each split
    for split_idx, vote_predictions in global_vote_predictions.items():
        global_metrics[split_idx] = eval_indiv_vote_predictions(vote_predictions, perf_metric)
    # Aggregate metrics for all splits
    global_metrics['global'] = {}
    for metric_type in global_metrics[0].keys():
        metrics_per_type = [metrics[metric_type] for split_key, metrics in global_metrics.items() if split_key != 'global']
        avgd_metrics_per_type = np.round(np.average(np.array(metrics_per_type), axis=0), 2)
        global_metrics['global'][metric_type] = avgd_metrics_per_type
    return global_metrics

### Query Times

In [75]:
def eval_indiv_query_times(query_times):
    times = [query_time for _, query_time in query_times.items()]
    avg_time = np.average(np.array(times))
    return avg_time

def eval_global_query_times(global_query_times):
    global_metrics = {}
    for split_idx, query_times in global_query_times.items():
        global_metrics[split_idx] = eval_indiv_query_times(query_times)
    avg_times = [avg_time for _, avg_time in global_metrics.items()]
    global_metrics['global'] = np.average(np.array(avg_times))
    return global_metrics

### Main Code

In [76]:
def evaluate_results(gbl_vote_preds, gbl_query_times, perf_metric):
    vote_metrics = eval_global_vote_predictions(gbl_vote_preds, perf_metric)
    query_metrics = eval_global_query_times(gbl_query_times)
    return {
        'vote_metrics': vote_metrics,
        'query_metrics': query_metrics
    }

In [77]:
def batch_evaluate_results(batch_gbl_vote_preds, batch_gbl_query_times, perf_metric):
    batch_results = {}
    for k_idx in batch_gbl_vote_preds.keys():
        results = evaluate_results(batch_gbl_vote_preds[k_idx], batch_gbl_query_times[k_idx], perf_metric)
        batch_results[k_idx] = results
    return batch_results

### Compute Both Vote Distances and Query Times

In [78]:
batch_global_results = batch_evaluate_results(batch_global_predictions, batch_global_query_times, perf_metric=PERF_METRIC_ID)

#### Display Results

In [79]:
def display_batch_results(batch_gbl_results, full_id):
    print('DISPLAYING RESULTS')
    print(f'FOR {full_id}')
    print('-------------------------')
    for k_idx, gbl_results in batch_gbl_results.items():
        seconds = gbl_results["query_metrics"]["global"]
        microseconds = round(seconds * 10**6)
        print(f'FOR K = {k_idx}')
        print(f'Avg. Accuracy : {gbl_results["vote_metrics"]["global"]}')
        print(f'Avg. Query Time : {microseconds} microseconds')
        print('-------------------------------------')

In [80]:
# Watch out! Query times can change!
display_batch_results(batch_global_results, FULL_ID)

DISPLAYING RESULTS
FOR color_hist_euclid
-------------------------
FOR K = 1
Avg. Accuracy : {'rmse': 4.42}
Avg. Query Time : 445 microseconds
-------------------------------------
FOR K = 3
Avg. Accuracy : {'rmse': 4.51}
Avg. Query Time : 484 microseconds
-------------------------------------
FOR K = 5
Avg. Accuracy : {'rmse': 4.51}
Avg. Query Time : 441 microseconds
-------------------------------------
FOR K = 7
Avg. Accuracy : {'rmse': 4.51}
Avg. Query Time : 468 microseconds
-------------------------------------
FOR K = 10
Avg. Accuracy : {'rmse': 4.51}
Avg. Query Time : 419 microseconds
-------------------------------------


Additional note: Average query times are similar when using both time and timeit.

In [81]:
tictoc

<function time.perf_counter>